In [1]:
import os 
import pandas as pd
import numpy as np

path = os.getcwd()

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path = 'C:\\Users\\sunil\\Projects\\Dockship\\segmind_grand_ai_challenge_2021-dataset'

In [3]:
train = pd.read_csv(path + "\\dataset\\TRAIN.csv")
test = pd.read_csv(path + "\\dataset\\TEST.csv")
ss = pd.read_csv(path + "\\dataset\\sample_submission.csv")
shop = pd.read_csv(path + "\\dataset\\store.csv")

C:\Users\sunil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
train['week'] = train['Date'].dt.week

test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['day'] = test['Date'].dt.day
test['week'] = test['Date'].dt.week

train['StateHoliday'].replace({0 : 4, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)
test['StateHoliday'].replace({0 : 4, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)

In [5]:
# Merging

train = pd.merge(train, shop, how = 'left', on = 'Store')
test = pd.merge(test, shop, how = 'left', on = 'Store')

train['Assortment'].unique()
train['Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)
test['Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)


a = [0, 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec']
b = [0, 1, 2, 3]

train['PromoInterval'].replace(dict(zip(a, b)), inplace = True)
test['PromoInterval'].replace(dict(zip(a, b)), inplace = True)

train['StoreType'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)
test['StoreType'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)


train['CompetitionDistance'].fillna(train['CompetitionDistance'].mean(), inplace = True)
test['CompetitionDistance'].fillna(test['CompetitionDistance'].mean(), inplace = True)

train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

In [6]:
target = 'Sales'
date = 'Date'
customer = 'Customers'

cat_cols = ['Store', 'DayOfWeek', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'year', 'month', 'day', 'dayofweek',
       'week', 'Store_Type', 'Store_Assortment',
       'Store_CompetitionOpenSinceMonth', 'Store_CompetitionOpenSinceYear',
       'Store_Promo2', 'Store_Promo2SinceWeek', 'Store_Promo2SinceYear',
       'Store_PromoInterval']

features = [col for col in train.columns if col not in [target, date, customer]]

In [7]:
trn, val = train_test_split(train, test_size = 0.2, random_state = 1)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test[features]

In [20]:
# Cross Validation for Boosting
def cross_val(regressor, train, test, features, name):
    N_splits = 5
    
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    target_col = train[target]
    
    folds = StratifiedKFold(n_splits = N_splits, shuffle = True,random_state = 1999)
    stratified_target = pd.qcut( train[target], 10, labels=False, duplicates='drop')
    for index, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
        print(f'\n================================Fold{index + 1}===================================')
        
        #### Train Set
        X_trn, y_trn = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
        
        #### Validation Set
        X_val, y_val = train[features].iloc[val_idx], train[target].iloc[val_idx]
        
        #### Test Set
        X_test = test[features]
        
        ############ Fitting #############
        _ = regressor.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 50, verbose = 100)
        
        ############ Predicting #############
        val_preds = np.abs(regressor.predict(X_val))
        test_preds = np.abs(regressor.predict(X_test))
        
        error = np.sqrt(mean_squared_error((y_val), (val_preds)))
        print(f'\n Root Mean Squared Error for Validation set is : {error}')
        
        oofs[val_idx] = val_preds
        preds += test_preds / N_splits
        
    total_error = np.sqrt(mean_squared_error((target_col), (oofs)))
    print(f'\n\Root Mean Squared Error for oofs is {total_error}')
    
    return oofs, preds

In [35]:
# Cross Validation for Boosting
def normal_cross_val(regressor, train, test, features, name):
    N_splits = 5
    
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    target_col = train[target]
    
    folds = StratifiedKFold(n_splits = N_splits, shuffle = True,random_state = 1999)
    stratified_target = pd.qcut( train[target], 10, labels=False, duplicates='drop')
    for index, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
        print(f'\n================================Fold{index + 1}===================================')
        
        #### Train Set
        X_trn, y_trn = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
        
        #### Validation Set
        X_val, y_val = train[features].iloc[val_idx], train[target].iloc[val_idx]
        
        #### Test Set
        X_test = test[features]
        
        ############ Fitting #############
        _ = regressor.fit(X_trn, y_trn)
        
        ############ Predicting #############
        val_preds = np.abs(regressor.predict(X_val))
        test_preds = np.abs(regressor.predict(X_test))
        
        error = np.sqrt(mean_squared_error((y_val), (val_preds)))
        print(f'\n Root Mean Squared Error for Validation set is : {error}')
        
        oofs[val_idx] = val_preds
        preds += test_preds / N_splits
        
    total_error = np.sqrt(mean_squared_error((target_col), (oofs)))
    print(f'\n\Root Mean Squared Error for oofs is {total_error}')
    
    return oofs, preds

In [9]:
def join(train_, test_):
    df = pd.concat([train_, test_], axis = 0).reset_index(drop=True)
    return df

def split(df):
    train_, test_ = df[:train.shape[0]], df[train.shape[0]:].reset_index(drop=True)
    return train_, test_

In [10]:
df = join(train, test)

df['Store_Count'] = df.groupby('Store')['Date'].transform('count')
df['Store_Cus_Count'] = df.groupby('Store')['Customers'].transform('sum')/df['Store_Count']
#df['Store_Mean_Sales'] = df.groupby('Store')['Sales'].transform('mean')
df['Day_of_week_Store_Sale'] = df.groupby(['Store', 'DayOfWeek', 'Promo'])['Sales'].transform('mean')
df['Open_Closed_Sales'] = df.groupby(['Store', 'Open'])['Sales'].transform('mean')
#df["CompetitionOpen"] = ((df["year"] - df["CompetitionOpenSinceYear"]) * 12 + (df["month"] - df["CompetitionOpenSinceMonth"]))

train_feat, test_feat = split(df)

test_feat['Open_Closed_Sales'].fillna(0, inplace=True)

In [11]:
# Changing Datatype

df = join(train_feat, test_feat)

int8_cols = ['DayOfWeek', 'Open', 'Promo','StateHoliday', 'SchoolHoliday', 
'month', 'day','week', 'StoreType', 'Assortment',
'CompetitionOpenSinceMonth','Promo2', 
'Promo2SinceWeek','PromoInterval']

int16_cols = ['year', 'Store', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'Store_Count', 'Store_Cus_Count']

int32_cols = ['CompetitionDistance', 'Day_of_week_Store_Sale', 'Open_Closed_Sales']

def int8(x):
    df[x] = df[x].astype('int8')

def int16(x):
    df[x] = df[x].astype('int16')

def int32(x):
    df[x] = df[x].astype('int32')

for col in int8_cols:
    int8(col)

for col in int16_cols:
    int16(col)

for col in int32_cols:
    int32(col)

train_feat, test_feat = split(df)

In [12]:
trn, val = train_test_split(train_feat, test_size = 0.2, random_state = 1)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test_feat[features]

In [26]:
xgb_params = {'n_estimators': 1485, 'learning_rate': 0.01073168998223091, 
              'max_depth': 9, 'colsample_bytree': 0.8822248508897588, 
              'subsample': 0.9983846370607781, 'min_child_weight': 1}

xgb = XGBRegressor(random_state = 1, tree_method = 'gpu_hist', **xgb_params)

In [16]:
lgb_params = {'n_estimators': 2590, 'learning_rate': 0.09823140324394083,
              'num_leaves': 238,'max_depth': 9, 'subsample': 0.9133061613972607}

lgb = LGBMRegressor(random_state=1, **lgb_params)

In [17]:
cat_params = {'n_estimators': 2908, 'learning_rate': 0.27751412081419524,
              'depth': 9, 'rsm': 0.9097284463105668}

cat = CatBoostRegressor(random_state=1, **cat_params)

In [18]:
cat_params = {'n_estimators': 2908, 'learning_rate': 0.27751412081419524,
              'depth': 9, 'rsm': 0.9097284463105668}

cat_2 = CatBoostRegressor(random_state=1, **cat_params, cat_features=['Store', 'Promo'])

In [29]:
%%time
cat_oofs, cat_preds = cross_val(cat, train_feat, test_feat, features, 'cat')


================================Fold1===================================
0:	learn: 3295.8958279	test: 3283.3519421	best: 3283.3519421 (0)	total: 79.6ms	remaining: 3m 51s
100:	learn: 1166.8460009	test: 1168.5858614	best: 1168.5858614 (100)	total: 7.62s	remaining: 3m 31s
200:	learn: 927.5599644	test: 935.9204676	best: 935.9204676 (200)	total: 14.6s	remaining: 3m 17s
300:	learn: 818.4870704	test: 834.1734383	best: 834.1734383 (300)	total: 21.5s	remaining: 3m 6s
400:	learn: 753.7699181	test: 776.2663731	best: 776.2663731 (400)	total: 28.5s	remaining: 2m 58s
500:	learn: 713.3858395	test: 743.1582655	best: 743.1582655 (500)	total: 35.5s	remaining: 2m 50s
600:	learn: 682.8520613	test: 718.4670616	best: 718.4670616 (600)	total: 42.5s	remaining: 2m 43s
700:	learn: 659.6623070	test: 701.9345239	best: 701.9345239 (700)	total: 49.6s	remaining: 2m 36s
800:	learn: 641.5142871	test: 689.0476380	best: 689.0476380 (800)	total: 56.6s	remaining: 2m 29s
900:	learn: 625.0038247	test: 677.8846409	best: 677

1700:	learn: 545.8213953	test: 623.0739702	best: 623.0739702 (1700)	total: 1m 58s	remaining: 1m 24s
1800:	learn: 539.0233091	test: 620.0449952	best: 620.0449952 (1800)	total: 2m 5s	remaining: 1m 17s
1900:	learn: 532.7196674	test: 617.2512048	best: 617.2512048 (1900)	total: 2m 12s	remaining: 1m 10s
2000:	learn: 526.1917795	test: 614.7114029	best: 614.7114029 (2000)	total: 2m 19s	remaining: 1m 3s
2100:	learn: 519.9905380	test: 612.3263643	best: 612.3263643 (2100)	total: 2m 26s	remaining: 56.2s
2200:	learn: 514.6204026	test: 610.4324838	best: 610.4324838 (2200)	total: 2m 32s	remaining: 49.1s
2300:	learn: 509.2920991	test: 608.8625052	best: 608.8558005 (2298)	total: 2m 39s	remaining: 42.2s
2400:	learn: 504.3770074	test: 607.2566485	best: 607.2566485 (2400)	total: 2m 46s	remaining: 35.2s
2500:	learn: 499.6377986	test: 605.5363386	best: 605.5326559 (2499)	total: 2m 53s	remaining: 28.2s
2600:	learn: 495.2119374	test: 604.2840985	best: 604.2840985 (2600)	total: 3m	remaining: 21.3s
2700:	learn:

In [30]:
%%time
xgb_oofs, xgb_preds = cross_val(xgb, train_feat, test_feat, features, 'xgb')


================================Fold1===================================
[0]	validation_0-rmse:6982.54346
[100]	validation_0-rmse:3137.16455
[200]	validation_0-rmse:2167.72437
[300]	validation_0-rmse:1869.05200
[400]	validation_0-rmse:1686.76514
[500]	validation_0-rmse:1532.13599
[600]	validation_0-rmse:1427.08411
[700]	validation_0-rmse:1325.19519
[800]	validation_0-rmse:1237.30054
[900]	validation_0-rmse:1162.02283
[1000]	validation_0-rmse:1103.91479
[1100]	validation_0-rmse:1053.30627
[1200]	validation_0-rmse:998.24475
[1300]	validation_0-rmse:959.94220
[1400]	validation_0-rmse:927.11713
[1484]	validation_0-rmse:901.82800

 Root Mean Squared Error for Validation set is : 901.8280280138662

================================Fold2===================================
[0]	validation_0-rmse:6996.58008
[100]	validation_0-rmse:3148.33032
[200]	validation_0-rmse:2173.80200
[300]	validation_0-rmse:1876.75427
[400]	validation_0-rmse:1694.23791
[500]	validation_0-rmse:1525.15601
[600]	validation

In [24]:
%%time
lgb_oofs, lgb_preds = cross_val(lgb, train_feat, test_feat, features, 'lgb')


================================Fold1===================================
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 1.48378e+06
[200]	valid_0's l2: 724222
[300]	valid_0's l2: 583487
[400]	valid_0's l2: 525736
[500]	valid_0's l2: 487077
[600]	valid_0's l2: 461895
[700]	valid_0's l2: 444507
[800]	valid_0's l2: 430915
[900]	valid_0's l2: 420300
[1000]	valid_0's l2: 410557
[1100]	valid_0's l2: 403377
[1200]	valid_0's l2: 396819
[1300]	valid_0's l2: 392050
[1400]	valid_0's l2: 387243
[1500]	valid_0's l2: 384016
[1600]	valid_0's l2: 380442
[1700]	valid_0's l2: 377278
[1800]	valid_0's l2: 374329
[1900]	valid_0's l2: 371921
[2000]	valid_0's l2: 369426
[2100]	valid_0's l2: 367350
[2200]	valid_0's l2: 365578
[2300]	valid_0's l2: 363708
[2400]	valid_0's l2: 362497
[2500]	valid_0's l2: 361387
Did not meet early stopping. Best iteration is:
[2590]	valid_0's l2: 360226

 Root Mean Squared Error for Validation set is : 600.0323445001073

========================

In [25]:
%%time
cat_2_oofs, cat_2_preds = cross_val(cat_2, train_feat, test_feat, features, 'cat_2')


================================Fold1===================================
0:	learn: 3079.1538325	test: 3048.0948726	best: 3048.0948726 (0)	total: 195ms	remaining: 9m 26s
100:	learn: 930.2205742	test: 874.9784136	best: 874.9784136 (100)	total: 22s	remaining: 10m 11s
200:	learn: 821.1389312	test: 786.5431949	best: 786.5431949 (200)	total: 42.2s	remaining: 9m 28s
300:	learn: 766.7888565	test: 743.7295258	best: 743.7295258 (300)	total: 1m 2s	remaining: 9m 2s
400:	learn: 727.5468057	test: 716.2638967	best: 716.2638967 (400)	total: 1m 24s	remaining: 8m 47s
500:	learn: 699.3835393	test: 696.7709272	best: 696.7709272 (500)	total: 1m 47s	remaining: 8m 35s
600:	learn: 676.9548910	test: 681.7984052	best: 681.7984052 (600)	total: 2m 9s	remaining: 8m 15s
700:	learn: 658.0699589	test: 670.4171328	best: 670.4171328 (700)	total: 2m 31s	remaining: 7m 57s
800:	learn: 641.5855067	test: 661.2239526	best: 661.2239526 (800)	total: 2m 54s	remaining: 7m 38s
900:	learn: 627.7656184	test: 652.7631862	best: 652.

1700:	learn: 552.0321550	test: 608.5587437	best: 608.5587437 (1700)	total: 6m 11s	remaining: 4m 23s
1800:	learn: 545.8151530	test: 605.8623529	best: 605.8623529 (1800)	total: 6m 31s	remaining: 4m
1900:	learn: 539.4812006	test: 603.2183666	best: 603.2161759 (1899)	total: 6m 52s	remaining: 3m 38s
2000:	learn: 533.9636565	test: 601.5259248	best: 601.5132270 (1998)	total: 7m 13s	remaining: 3m 16s
2100:	learn: 528.0594485	test: 599.8160860	best: 599.8160860 (2100)	total: 7m 33s	remaining: 2m 54s
2200:	learn: 522.8309939	test: 598.4230226	best: 598.4169853 (2199)	total: 7m 53s	remaining: 2m 32s
2300:	learn: 517.6549884	test: 596.6448822	best: 596.6448822 (2300)	total: 8m 13s	remaining: 2m 10s
2400:	learn: 512.4748186	test: 595.0564665	best: 595.0564665 (2400)	total: 8m 33s	remaining: 1m 48s
2500:	learn: 507.8384709	test: 593.7634475	best: 593.7634475 (2500)	total: 8m 56s	remaining: 1m 27s
2600:	learn: 503.4219554	test: 592.2831090	best: 592.2831090 (2600)	total: 9m 19s	remaining: 1m 6s
2700:

In [31]:
test[target] = None

train_new = train[[target, 'Store']].copy()
test_new = test[[target, 'Store']].copy()

train_new['lgb_1'] = lgb_oofs
test_new['lgb_1'] = lgb_preds

# train_new['lgb_2'] = lgb_k_oofs
# test_new['lgb_2'] = lgb_k_preds

train_new['xgb'] = xgb_oofs
test_new['xgb'] = xgb_preds

train_new['cat'] = cat_oofs
test_new['cat'] = cat_preds

train_new['cat_2'] = cat_2_oofs
test_new['cat_2'] = cat_2_preds

In [32]:
ens_features = [c for c in train_new.columns if c not in [target, 'Store']]

In [36]:
%%time
level_1_lgb_oofs, level_1_lgb_preds = normal_cross_val(LinearRegression(), train_new, test_new, ens_features, 'lr')


================================Fold1===================================

 Root Mean Squared Error for Validation set is : 577.2279463130423

================================Fold2===================================

 Root Mean Squared Error for Validation set is : 573.8146257097959

================================Fold3===================================

 Root Mean Squared Error for Validation set is : 567.5084094599686

================================Fold4===================================

 Root Mean Squared Error for Validation set is : 585.7644072250088

================================Fold5===================================

 Root Mean Squared Error for Validation set is : 574.4900052642206

\Root Mean Squared Error for oofs is 575.7916011962312
Wall time: 1.04 s


In [37]:
test['Predictions'] = level_1_lgb_preds
test.loc[ test['Open'] == 0, 'Predictions'] = 0
preds = test['Predictions']

index = [i for i in range(test.shape[0])]

d = list(zip(index, preds))

ss = pd.DataFrame(d, columns = ['index', 'Sales'])

ss.to_csv(path + "\\a.csv", index = False)